<a href="https://colab.research.google.com/github/yaniv92648/ITC_Final_Project_Personality_Types/blob/master/ITC_Final_Project_Personality_Types.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression

In [2]:
data = pd.read_csv('MBTI.csv.zip', compression='zip')

In [3]:
from copy import deepcopy
data_copy = deepcopy(data)

In [4]:
data

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...
...,...,...
8670,ISFP,'https://www.youtube.com/watch?v=t8edHB_h908||...
8671,ENFP,'So...if this thread already exists someplace ...
8672,INTP,'So many questions when i do these things. I ...
8673,INFP,'I am very conflicted right now when it comes ...


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8675 entries, 0 to 8674
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    8675 non-null   object
 1   posts   8675 non-null   object
dtypes: object(2)
memory usage: 135.7+ KB


In [6]:
data.describe()

,type,posts
count,8675,8675
unique,16,8675
top,INFP,"'INTP?? I don't know, I think he's hard to typ..."
freq,1832,1


In [7]:
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import spacy
import re

In [8]:
# First time only
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package wordnet to /Users/mac/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mac/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
def preprocess_data():
    
    stop_words = stopwords.words("english")
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    nums_of_videos = []
    nums_of_images = []
    posts = []
    
    for i in range(len(data.posts)):
        
        post = data.posts[i]
        
        urls = re.findall(url_regex, post)
        num_of_images = len([url for url in urls if url.endswith('png') or url.endswith('jpg') or url.endswith('gif')])
        num_of_videos = len(urls) - num_of_images
        nums_of_videos.append(num_of_videos)
        nums_of_images.append(num_of_images)

        # Remove urls
        post = re.sub(url_regex, '', post)

        # Keep only words
        post = re.sub("[^a-zA-Z]", " ", post)

        # Remove spaces > 1
        post = re.sub(' +', ' ', post).lower()

        # Lemmatization & remove stop words (common useless words such as "if" then" "and" "or" etc..)
        post = nlp(post)
        post = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' and word.text not in stop_words 
                                     else word.text 
                         for word in post])
        
        posts.append(post)
        
    
    data['num_of_images'] = nums_of_images
    data['num_of_videos'] = nums_of_videos
    data['posts'] = posts
    

In [10]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize

In [11]:
def perform_TF_IDF():
    vectorizer = TfidfVectorizer()
    matrix = vectorizer.fit_transform(data.posts)
    matrix = matrix.todense()
    return pd.DataFrame(matrix, columns=vectorizer.get_feature_names())

In [12]:
data = deepcopy(data_copy)
data['unique_target_name_personality_type'] = data['type']
data.drop(columns='type', inplace=True)

In [13]:
preprocess_data()

In [14]:
data_after_preprocess = deepcopy(data)

In [15]:
final_data = perform_TF_IDF()
final_data['unique_target_name_personality_type'] = data['unique_target_name_personality_type']

In [16]:
from sklearn.model_selection import train_test_split
X = final_data.drop(columns='unique_target_name_personality_type')
y = final_data['unique_target_name_personality_type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

/opt/anaconda3/envs/LearningPython/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [18]:
y_pred = log_reg.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        ENFJ       0.43      0.07      0.12        41
        ENFP       0.77      0.56      0.65       125
        ENTJ       0.82      0.20      0.33        44
        ENTP       0.71      0.50      0.58       135
        ESFJ       0.00      0.00      0.00         7
        ESFP       0.00      0.00      0.00         8
        ESTJ       0.00      0.00      0.00         7
        ESTP       0.00      0.00      0.00        15
        INFJ       0.63      0.67      0.65       288
        INFP       0.55      0.88      0.68       370
        INTJ       0.61      0.69      0.65       193
        INTP       0.64      0.81      0.71       293
        ISFJ       1.00      0.13      0.24        45
        ISFP       0.57      0.08      0.13        53
        ISTJ       0.75      0.07      0.12        44
        ISTP       0.83      0.36      0.50        67

    accuracy                           0.62      1735
   macro avg       0.52   

/opt/anaconda3/envs/LearningPython/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
feature_importance = abs(log_reg.coef_[0])
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = list(np.argsort(feature_importance))
features = list(final_data.columns)
features_idx = list(zip(features, sorted_idx))
features_idx.sort(key=lambda tup: tup[1], reverse=True)
np.save('feature_importance', features_idx)

In [34]:
len(features_idx)

84822

In [ ]:
params = {'n_estimators': [10], 'criterion': ['gini', 'entropy']}

In [ ]:
clf = RandomForestClassifier(n_estimators=500)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
clf = AdaBoostClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))